# Analyzing the Data

# Data preparation

In [35]:
#import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


import ROOT;
#import lumiere as lm
#lm.loadstyle(True);

from sklearn.metrics import roc_auc_score, roc_curve

def ams_score(x, y, w, cut):
# Calculate Average Mean Significane as defined in ATLAS paper
#    -  approximative formula for large statistics with regularisation
# x: array of truth values (1 if signal)
# y: array of classifier result
# w: array of event weights
# cut
    t = y > cut 
    s = np.sum((x[t] == 1)*w[t])
    b = np.sum((x[t] == 0)*w[t])
    return s/np.sqrt(b+10.0)

def find_best_ams_score(x, y, w):
# find best value of AMS by scanning cut values; 
# x: array of truth values (1 if signal)
# y: array of classifier results
# w: array of event weights
#  returns 
#   ntuple of best value of AMS and the corresponding cut value
#   list with corresponding pairs (ams, cut) 
# ----------------------------------------------------------
    ymin=min(y) # classifiers may not be in range [0.,1.]
    ymax=max(y)
    nprobe=200    # number of (equally spaced) scan points to probe classifier 
    amsvec= [(ams_score(x, y, w, cut), cut) for cut in np.linspace(ymin, ymax, nprobe)] 
    maxams=sorted(amsvec, key=lambda lst: lst[0] )[-1]
    return maxams, amsvec




def printScore(model):

    try:
        pred_clf = model.predict_proba(x_val)[:, 1]
    except:
        pred_clf = model.predict(x_val)
        pred_clf = pred_clf.reshape((pred_clf.shape[0],))

    auc = roc_auc_score(y_val, pred_clf, sample_weight=w_val)
    print('AUC:', auc)
    bs = find_best_ams_score(y_val, pred_clf, w_val)
    print('AMS:', bs[0][0])
    print('AMS total:', bs[0][0]*np.sqrt(50))



def plotLossAccuracy(history):
    # Get training and validation loss/accuracy values from history
    loss_training = history.history['loss']
    loss_validation = history.history['val_loss']
    accuracy_training = history.history['accuracy']
    accuracy_validation = history.history['val_accuracy']


    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20, 8))
    
    ax1.plot(loss_training, '-', label='training')
    ax1.plot(loss_validation, '-', label='validation')
    ax1.legend(title='loss')
    ax1.set_xlabel('epoch')
    
    ax2.plot(accuracy_training, '-', label='training')
    ax2.plot(accuracy_validation, '-', label='validation')
    ax2.legend(title='accuracy')
    ax2.set_xlabel('epoch')

    plt.show()


def plotAMS(history):
    # Get training and validation loss/accuracy values from history
    ams_training = history.history['ams_metric']
    ams_validation = history.history['val_ams_metric']
    

    fig, ax1 = plt.subplots(1,1, figsize=(10, 8))
    
    #ax1.plot(ams_training, 'o--', label='training')
    ax1.plot(ams_validation, '-', label='validation')
    ax1.legend(title='AMS')
    ax1.set_xlabel('epoch')
    
    plt.show()

## Read-in & to Pandas

In [36]:

input_columns = ['DER_deltaeta_jet_jet', 'DER_deltar_tau_lep', 'DER_lep_eta_centrality', 'DER_mass_MMC', 'DER_mass_jet_jet', 
                 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_met_phi_centrality', 'DER_prodeta_jet_jet', 'DER_pt_h', 
                 'DER_pt_ratio_lep_tau', 'DER_pt_tot', 'DER_sum_pt', 'PRI_jet_all_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 
                 'PRI_jet_leading_pt', 'PRI_jet_num', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_subleading_pt', 
                 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_lep_pt', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet', 'PRI_tau_eta', 'PRI_tau_phi', 
                 'PRI_tau_pt', 'transverse_lepton_jet_mass']


'''

input_columns = [ 'transverse_lepton_jet_mass',
                'DER_mass_MMC', 
                'DER_mass_vis', 
                'DER_mass_transverse_met_lep',
                'DER_pt_ratio_lep_tau', 
                'DER_sum_pt',
                'DER_deltar_tau_lep', 
                'DER_pt_h',
                'DER_sum_pt',  
                'PRI_met_sumet', 
                'PRI_tau_pt',
                ]

'''
print(len(input_columns))

31


In [37]:
RDF = ROOT.ROOT.RDataFrame

signal_tree_name = 'signal'
background_tree_name = 'background'
test_tree_name = 'validation'
file_name = 'atlas-higgs-challenge-2014-v2_part.root'

rdf_signal = RDF(signal_tree_name, file_name)
rdf_bkg = RDF(background_tree_name, file_name)
rdf_test = RDF(test_tree_name, file_name)

reconstruct_transverse_lepton_jet_mass = '''

float lep_px = PRI_lep_pt * TMath::Cos(PRI_lep_phi);
float lep_py = PRI_lep_pt * TMath::Sin(PRI_lep_phi);
float jet_px = PRI_jet_leading_pt * TMath::Cos(PRI_jet_leading_phi);
float jet_py = PRI_jet_leading_pt * TMath::Sin(PRI_jet_leading_phi);

//calculate angle between jet and lepton
float cos_theta = (lep_px*jet_px + lep_py*jet_py) / PRI_lep_pt / PRI_jet_leading_pt;

return PRI_lep_pt * PRI_jet_leading_pt * (1 - cos_theta);
'''

#insertion
rdf_signal = rdf_signal.Define('transverse_lepton_jet_mass', reconstruct_transverse_lepton_jet_mass)
rdf_bkg = rdf_bkg.Define('transverse_lepton_jet_mass', reconstruct_transverse_lepton_jet_mass)
rdf_test = rdf_test.Define('transverse_lepton_jet_mass', reconstruct_transverse_lepton_jet_mass)

# label classification to int values
rdf_test = rdf_test.Define('IntLabel', '''
const char ch = Label[0];
const char s = 's';
if(ch == s){
    return 1;
}
else{
    return 0;
}
''')


df_signal = pd.DataFrame(rdf_signal.AsNumpy())
df_bg = pd.DataFrame(rdf_bkg.AsNumpy())
df_test = pd.DataFrame(rdf_test.AsNumpy())


In [38]:
#df_signal.to_csv('data/signal_df.csv', index=False)
#df_bg.to_csv('data/bg_df.csv', index=False)
#df_test.to_csv('data/test_df.csv', index=False)

## concatination, shuffle and split

In [39]:
from sklearn.utils import shuffle;
from sklearn.model_selection import train_test_split;

#input feature arrays
vars_signal = df_signal[input_columns].to_numpy()
vars_bg = df_bg[input_columns].to_numpy()
vars_test = df_test[input_columns].to_numpy()

inputs = np.concatenate([vars_signal, vars_bg])

#weights
weight_signal = df_signal['Weight'].to_numpy()
weight_bg = df_bg['Weight'].to_numpy()
weights = np.concatenate([weight_signal, weight_bg])
weights = weights.reshape((weights.shape[0],))

weights_test = df_test['Weight'].to_numpy()


# target classifictionation (1:signal / 0: background)
y_signal = np.ones((vars_signal.shape[0], ))
y_bg = np.zeros((vars_bg.shape[0], ))

targets = np.concatenate([y_signal, y_bg])

# for test dataset there is already a classification; convert to int
truths_test = df_test.IntLabel.to_numpy()


# shuffle 
inputs, targets, weights = shuffle(inputs, targets, weights)


# not for gridcv

# training and validation split  (80, 20)
x_train, x_val, y_train, y_val, w_train, w_val = train_test_split(inputs, targets, weights, test_size=0.2)
#x_train, y_train = inputs, targets

## StandardScaling 

In [40]:
from sklearn.preprocessing import StandardScaler;
 
scaler = StandardScaler()
scaler.fit(x_train) #set up only on train data
 
# tranformation applied to all
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(vars_test)

### SMOTE

In [41]:
#%pip install imbalanced-learn

In [42]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)


In [43]:
#from imblearn.over_sampling import ADASYN
#ada = ADASYN(random_state=0)
#x_train, y_train = ada.fit_resample(x_train, y_train)

## PCA

In [44]:
from sklearn.decomposition import PCA

x_train_pre = x_train

#pca = PCA(n_components=22)
#pca.fit(x_train)

#x_train = pca.transform(x_train)
#x_val = pca.transform(x_val)
#x_test = pca.transform(x_test)

## DNN

$$
AMS = \sqrt{2((s+b+b_r) \log(1+\frac{s}{s+b_r})-s)}
$$

Here:
- $s, b$: unnormalised true positive and false positive rates, respectively
- $b_r = 10$ is the constant regularisation term


In [45]:
from tensorflow.keras.metrics import Metric
from tensorflow.keras.metrics import AUC


def ams_metric_exact(y_true, y_pred):
    
    s = tf.reduce_sum(y_true * tf.round(y_pred))
    b = tf.reduce_sum((1 - y_true) * tf.round(y_pred))
    b_r = 10.0
    ams = tf.math.sqrt(2 * ((s + b + b_r) * tf.math.log(1 + s / (b + b_r)) - s))
    return ams.numpy()

def ams_scorer_wrapper(w):
    def my_find_best_ams_score(x, y):
        ymax=max(y)
        nprobe=200    # number of (equally spaced) scan points to probe classifier 
        amsvec= [(ams_score(x, y, w, cut), cut) for cut in np.linspace(ymin, ymax, nprobe)] 
        maxams=sorted(amsvec, key=lambda lst: lst[0] )[-1]
        return maxams, amsvec
    return my_find_best_ams_score


#### training function

In [75]:

# Training function
def train(model, epochs=10, batch_size=128, learning_rate=1e-3, verbose="auto", callbacks=None):
    
    
    # Define loss function, optimizer algorithm and validation metrics
    model.compile(
        loss='binary_crossentropy',
        #optimizer=Adam(learning_rate=learning_rate, weight_decay=0.1),
        metrics=['accuracy', AUC],
    )
    
    # Print summary of the model
    model.summary()
    
    # Train model

    #currently including the weights th oest the new metric
    #history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
    #                     verbose=verbose, callbacks=callbacks, validation_data=(x_val, y_val))
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                         verbose=verbose,  validation_split=0.2)
        
    
    return history


### model

In [77]:
import tensorflow as tf;
from tensorflow.keras import layers, models, optimizers;
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping


# Define the DNN model
def buildModel():
    model = models.Sequential()
    
    # input
    model.add(layers.Input(shape=(x_train.shape[1],)))
    
    # hidden
    model.add(layers.Dense(2*128, activation='relu', kernel_initializer='glorot_normal'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(2*128, activation='relu', kernel_initializer='glorot_normal'))
    model.add(layers.Dropout(0.4))    
    model.add(layers.Dense(2*128, activation='relu', kernel_initializer='glorot_normal'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(128, activation='sigmoid', kernel_initializer='glorot_normal'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(128, activation='relu', kernel_initializer='glorot_normal'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(32, activation='relu', kernel_initializer='glorot_normal'))
    
    
    # output
    model.add(layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal'))
    
    return model


## execution

In [76]:
name = 'new_sigmoid_auc_luck'

model = buildModel()

#stopper = EarlyStopping(monitor="val_accuracy",
 #                       patience=20,
  #                      mode="max",
   #                     restore_best_weights=True,
    #                    start_from_epoch=100
     #                  )

#callbacks = [ModelCheckpoint(filepath=f'storage/{name}.keras', save_best_only=True), 
 #            CSVLogger(f'storage/{name}_training.log'),
  #          stopper]
        
#            EarlyStopping(patience=100)
 

history = train(model, epochs=400, batch_size=4*1024, learning_rate=2e-4)
#history = train(model, epochs=400, batch_size=2*1024, learning_rate=2e-3)

printScore(model)
plotLossAccuracy(history)


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_105 (Dense)               │ (None, 256)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_75 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_106 (Dense)               │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_76 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_77 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_108 (Dense)               │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_78 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_109 (Dense)               │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_79 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_110 (Dense)               │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_111 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 193,345 (755.25 KB)

 Trainable params: 193,345 (755.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/400
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 240ms/step - accuracy: 0.5354 - auc: 0.5149 - loss: 0.7289 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_loss: 1.0403
Epoch 2/400
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.6379 - auc: 0.6618 - loss: 0.6237 - val_accuracy: 0.5615 - val_auc: 0.0000e+00 - val_loss: 0.8605
Epoch 3/400
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.6997 - auc: 0.7611 - loss: 0.5595 - val_accuracy: 0.7469 - val_auc: 0.0000e+00 - val_loss: 0.6506
Epoch 4/400
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - accuracy: 0.7383 - auc: 0.7902 - loss: 0.5320 - val_accuracy: 0.6910 - val_auc: 0.0000e+00 - val_loss: 0.7069
Epoch 5/400
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - accuracy: 0.7586 - auc: 0.8130 - loss: 0.5073 - val_accuracy: 0.7990 - val_auc: 0.0000e+00 - val_loss: 0.4939
Epoch 6/400
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - accuracy: 0.7646 - auc: 0.8226 - loss: 0.4979 - val_accuracy: 0.7222 - val_auc: 0.0000e+00 - val_loss: 0.6286
Epoch 

KeyboardInterrupt: 

In [78]:
othermodel = buildModel()

othermodel.load_weights('storage/model_sigmoid_with_auc_scorer_weights_2.6ams.weights.h5')
#othermodel.load_weights('storage/model_sigmoid_weights_2.6ams.weights.h5')

In [79]:
printScore(othermodel)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
AUC: 0.9406685344699556
AMS: 0.4287342199811543
AMS total: 3.031608742753992


In [ ]:
#model.save_weights(f'storage/weights of {name}.weights.h5')

In [102]:
from keras import saving

test = saving.load_model('storage/model_b.keras')

In [103]:
printScore(test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
AUC: 0.9424497987969678
AMS: 0.44997116522228015
AMS total: 3.181776622670867


In [104]:
test.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 580,037 (2.21 MB)

 Trainable params: 193,345 (755.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 386,692 (1.48 MB)